In [ ]:
#! /usr/bin/env python

"""Generates the IDCTAB. 

This script serves as a pipeline to create the IDCTAB from various 
other inputs and is a wrapper around several modules that perform 
subtasks of the IDCTAB creation algorithm.

Author(s)
---------
    Catherine Martlin, January 2017.

Use
--- 

python make_idctab.py --filter_name='F606W' --filename_list='f606w_oct_2016_flc_files_removed_mismatched_odd_file_removed.txt' --path_to_data='/grp/hst/wfc3h/verap/F606W_FLC/xymc/' --path_to_coeff_files='/grp/hst/wfc3h/verap/F606W_FLC/meta_pix.v32_new/'


python make_idctab.py --filter_name='F410M' --filename_list='F410M_oldcat_filenames.txt' 
--path_to_data='/grp/hst/wfc3h/verap/CAL_14393/F410M_oldcat/xymc/' 
--path_to_coeff_files='/grp/hst/wfc3h/verap/CAL_14393/F410M_oldcat/meta_pix.v3/'

Dependencies
------------

Notes
-----
"""


import argparse
from troll import troll_main
#from coeff2v23_rot2mass import coeff2v23_rot2mass_main
from coeff2v23_rot2mass_vanLeeuween_update_theta import coeff2v23_rot2mass_vanLeeuween_update_theta_main
from veracoeff_scaling import veracoeff_scaling_main
from readwf3poly_onefilter import readwf3poly_onefilter_main
import datetime
import numpy as np
# ---------------------------------------------------------------------
def prep_files_for_readwf3poly(veracoeff_output, coeff2v23_output):
    """

        """

    list_of_values = []
    veracoeff_output_correct_order = [veracoeff_output[1],veracoeff_output[0]]

    for i in veracoeff_output_correct_order:
        with open(i,'r') as output:
            for line in output:
                line = line.strip('\n')
                list_of_values.append(line)

    print(list_of_values)
    for j in coeff2v23_output:
        list_of_values.remove(j)
    print(list_of_values)

    if 'vafactor' in i:
        outfile_name = 'veracoeff_combined_output_vafactor.txt'
    else: 
        outfile_name = 'veracoeff_combined_output.txt'
    f = open(outfile_name, 'w')
    for k in list_of_values:
        print(k)
        f.write(k + '\n')
    f.close()

    return outfile_name

# ---------------------------------------------------------------------
# The main controller
# ---------------------------------------------------------------------

def make_idctab_main(filter_name, filename_list, path_to_data, path_to_coeff_files):
    """ The main controller of the make_idctab_main

        Parameters
        ----------

        Returns
        -------

        """
    outfile_date = datetime.date.today()
    extension_list = [1,4]
    uvis_list = ['uvis1','uvis2']

    screen_outputfile = filter_name + '_'+ str(outfile_date) + '_outputs.info'

    troll_outputs = []
    print( " ")
    print("RUNNING troll.py:")
    for i in extension_list:
        print(screen_outputfile)
        print(path_to_coeff_files)
        troll_main(filename_list, outfile_date, i, path_to_data,path_to_coeff_files,screen_outputfile)
        troll_outfile = 'troll_output_{}_{}.txt'.format(i,outfile_date)
        troll_outputs.append(troll_outfile)
        print(troll_outfile)

    uvis_names = ['uvis2','uvis1']
    uvis_troll = zip(troll_outputs,uvis_names)

    coeff2v23_output = []
    print(" ")
    print("RUNNING coeff2v23_rot2mass.py:")
    for k in uvis_troll:
        print(k)
        uvis = k[1]
        troll_output = k[0]
        #if vanL == 'True': 
        coeff2v23_rot2mass_vanLeeuween_update_theta_main(uvis,filter_name,outfile_date,troll_output,path_to_coeff_files,screen_outputfile)
        #coeff2v23_rot2mass_vanLeeuween_main(uvis,filter_name,outfile_date,troll_output,path_to_coeff_files,screen_outputfile)
        #coeff_outfile_name = 'coeff2v23_with_vafactor_output_vL_mean_theta_{}_{}_{}.txt'.format(uvis, filter_name, outfile_date)
        
        # If you want to use the new_cat theta then we need
        # to go into coeff2v23... and change the theta value 
        # being used and uncomment line 111 and comment 116. 
        coeff_outfile_name = 'coeff2v23_vafactor_output_vL_mean_old_cat_theta_{}_{}_{}.txt'.format(uvis, filter_name, outfile_date)
        print(coeff_outfile_name)
        coeff2v23_output.append(coeff_outfile_name)
            
    chip_number = ['2','1']
    chip_coeff2v23 = zip(chip_number,coeff2v23_output)
    veracoeff_output = []
    veracoeff_output_vafactor = []
    print(" ")
    print("RUNNING veracoeff.py:")
    for l in chip_coeff2v23:
        print(chip_coeff2v23)
        chipnumber = l[0]
        infile = l[1]
        veracoeff_scaling_main(infile, chipnumber,outfile_date,screen_outputfile)
        detector = "UVIS" + str(chipnumber)
        outfile_veracoeff = "veracoeff_output_{}_{}.txt".format(detector,outfile_date)
        outfile_veracoeff_vafactor = "veracoeff_output_vafactor_{}_{}.txt".format(detector,outfile_date)
        print(outfile_veracoeff)
        print(outfile_veracoeff_vafactor)
        veracoeff_output.append(outfile_veracoeff)
        veracoeff_output_vafactor.append(outfile_veracoeff_vafactor)

    #Prep output of veracoeff for readwf3poly:
    combined_veracoeff_output = prep_files_for_readwf3poly(veracoeff_output, coeff2v23_output)
    combined_veracoeff_output_vafactor = prep_files_for_readwf3poly(veracoeff_output_vafactor, coeff2v23_output)
    print(" ")

    readwf3poly_outfile = 'UVIS_{}_CM_{}_idc.txt'.format(filter_name,outfile_date)
    readwf3poly_outfile_vafactor = 'UVIS_vafactor_{}_CM_{}_idc.txt'.format(filter_name,outfile_date)
    print(" ")
    print("RUNNING readwf3poly_perfilter.py:")
    readwf3poly_onefilter_main(combined_veracoeff_output,readwf3poly_outfile,screen_outputfile)
    print(combined_veracoeff_output)
    readwf3poly_onefilter_main(combined_veracoeff_output_vafactor,readwf3poly_outfile_vafactor,screen_outputfile)
    print(combined_veracoeff_output_vafactor)

    print(" ")
    print("This IDCTAB was found using the VanLeeuween rotations and the mean old catalogue theta value.")



In [ ]:
#! /usr/bin/env python
'''
ABOUT:


--filename_list - A list of the paths to the files in question.

--path_to_data (Optional)- The directory path to the files.

-- outfile - Name of file the products are printed to.

-- extension - UVIS chip to run on. Either 1 or 4 for UVIS 2 or 1.

EXAMPLE RUN:

python troll.py --filename_list --outfile_date --extension

python troll.py -fl=all_filenames.txt -out_date=mar_31_2016 -ext=4
'''

#import all the python things here!
import math
from astropy.io import fits
import os
import argparse
import numpy as np
import sys

# -----------------------------------------------------------------------------
def upload_header_info(flt_file, extension, path_to_data):
    '''
    We want to get these values from the header instead of looking
    them up and plugging them in when we run the code.

    dec0 - The DEC_TARG or delcination of the
            target that is found in the header.

    roll0 - The PA_V3 value, or the angle between
            North and the V3 vector, that is found in
            the header.

    '''

    flt_path = path_to_data + flt_file

    hdu = fits.open(flt_path)
    dec0 = hdu[0].header['DEC_TARG']
    roll0 = hdu[0].header['PA_V3']
    ra0 = hdu[0].header['RA_TARG']
    if extension == 4:
        orientat_header = hdu[4].header['ORIENTAT']
        crval1 = hdu[4].header['crval1']
        crval2 = hdu[4].header['crval2']
        v_factor0 = hdu[4].header['VAFACTOR']
    elif extension == 1:
        orientat_header = hdu[1].header['ORIENTAT']
        crval1 = hdu[1].header['crval1']
        crval2 = hdu[1].header['crval2']
        v_factor0 = hdu[1].header['VAFACTOR']
    hdu.close()

    return dec0,roll0,ra0,crval1,crval2,orientat_header,v_factor0
# -----------------------------------------------------------------------------
def convert_to_rads(item):
    '''
    The trigonometric functions take in radians so here we convert
    from arcseconds into radians.
    '''

    item_degree = item * (1./3600.)
    item_rad = math.radians(item_degree)

    return item_rad
# -----------------------------------------------------------------------------
def read_in_angle_stat(path, path_to_angle, extension):
    '''
    Read in the values from angle.stat and match to the file being
    calculated and provide the calculated orientat and the EPS for
    that file. Make sure the angle.stat file is correct for whichever
    UVIS chip you are using.
    '''
    # Cut out the unique name of the file from the path:

    length = len(path)
    mid = length - 11 - 7
    end = length - 7
    end2 = length - 9
    current_I = path[:end2]

    name_list = []
    orientat_calc_list = []
    EPS_list = []
    angle_file = path_to_angle + "angle.stat"

    orientat_calc = 0.0
    EPS = 0.0

    # Read in all the angle.stat info and match to the unique name.
    print(extension)
    if extension == 1:
        with open(angle_file, 'r') as angle_stats_1:
            for line in angle_stats_1:
                curr_name = line[0:9]
                extension_val = line[10:11]
                curr_orientat = line[21:32]
                curr_EPS = line[38:47]
                name_list.append(curr_name)
                orientat_calc_list.append(curr_orientat)
                EPS_list.append(curr_EPS)

            for x in range(len(name_list)):
                if name_list[x] == current_I:
                    orientat_calc = orientat_calc_list[x]
                    EPS = EPS_list[x]

    elif extension == 4:
        with open(angle_file, 'r') as angle_stats_4:
            for line in angle_stats_4:
                curr_name = line[0:9]
                extension_val = line[10:11]
                curr_orientat = line[21:32]
                curr_EPS = line[38:47]
                name_list.append(curr_name)
                orientat_calc_list.append(curr_orientat)
                EPS_list.append(curr_EPS)

            for x in range(len(name_list)):
                if name_list[x] == current_I :
                    orientat_calc = orientat_calc_list[x]
                    EPS = EPS_list[x]


    return orientat_calc, EPS
# -----------------------------------------------------------------------------
# The main controller
# -----------------------------------------------------------------------------

def troll_main(filename_list, outfile_date, extension, path_to_data,path_to_coeffs,screen_outputfile):
    '''
    The main controller/conversion tool.
    '''
    orig_stdout = sys.stdout
    out_f = open(screen_outputfile, 'a')
    sys.stdout = out_f

    with open(filename_list, 'r') as f:
        for path in f:
            path = path.rstrip()
            # Get the orientat_calc and EPS from the angle.stat:
            path_to_angle = path_to_coeffs
            #path_to_angle = '/grp/hst/wfc3o/martlin/idctab_vera/make_idctab_codes/'
            print(path)
            orientat_calc, EPS = read_in_angle_stat(path, path_to_angle, extension)

            # Read in the files and the values
            dec0, roll0, ra0, crval1, crval2, orientat_header, v_factor0 = upload_header_info(path, extension, path_to_data)

            # Convert the stuff:
            pi = math.pi
            v2 = -27.4596 # New one from 2012
            v3 = -33.2604
            v2_rad = convert_to_rads(v2)
            v3_rad = convert_to_rads(v3)

            # Now work out the roll:
            v2_sin = math.sin(v2_rad)**2
            v3_sin = math.sin(v3_rad)**2
            v2_sin1 = math.sin(v2_rad)
            v3_sin1 = math.sin(v3_rad)
            sin_rho_sqr = v2_sin + v3_sin - v2_sin*v3_sin

            if sin_rho_sqr > 0.0:
                sin_rho = math.sqrt(sin_rho_sqr)
                cos_rho = math.sqrt(1.0 - sin_rho_sqr)
                if v2 > 0.0:
                    beta = math.asin(v3_sin1/sin_rho)
                else:
                    beta = pi - math.asin(v3_sin1/sin_rho)
                if v3 > 0.0:
                    gamma = math.asin(v2_sin1/sin_rho)
                else:
                    gamma = pi - math.asin(v2_sin1/sin_rho)

                angle_a = pi/2.0 + math.radians(roll0) - beta
                sin_a = math.sin(angle_a)
                cos_a = math.cos(angle_a)
                dec0_rad = math.radians(dec0)
                cos_dec0 = math.cos(dec0_rad)
                sin_dec0 = math.sin(dec0_rad)

                angle_b = math.atan2(sin_a * cos_dec0, sin_dec0 *sin_rho - cos_dec0*cos_rho*cos_a)

                roll_rad = pi - (gamma + angle_b)
                if roll_rad < 0.0:
                    roll_rad = roll_rad + 2.0*pi # To keep roll between +/- pi
                roll_deg = math.degrees(roll_rad)
            else:
                roll_deg = roll0


            # Then print all the things to a file?
            outfile = 'troll_output_{}_{}.txt'.format(extension,outfile_date)
            f = open(outfile, 'a')
            filename = path + '[' + str(extension) + ']'
            ra0_prec = '{:}'.format(np.float64(ra0)) #{:.6e}
            dec0_prec = '{:}'.format(np.float64(dec0)) #{:.6e}
            crval1_prec = '{:}'.format(np.float64(crval1)) #{:.6e}
            crval2_prec = '{:}'.format(np.float64(crval2)) #{:.6e}
            orientat_header_prec = '{:}'.format(np.float64(orientat_header)) #{:.6e}
            orientat_calc_prec = '{:}'.format(np.float64(orientat_calc)) #{:.6e}
            EPS_prec = '{:}'.format(np.float64(EPS)) #{:.6e}
            roll0_prec = '{:}'.format(np.float64(roll0)) #{:.8e}
            roll_deg_prec = '{:}'.format(np.float64(roll_deg)) #{:.8e}
            v_factor0_prec = '{:}'.format(np.float64(v_factor0)) #{:.6e}
            allangle = filename + ',' + str(ra0) + ',' + str(dec0) + ',' + str(crval1) + ',' + str(crval2) + ',' + \
                        str(orientat_header) + ',' + str(orientat_calc_prec) + ',' + str(EPS_prec) + ',' + str(roll0_prec) + ',' + \
                        str(roll_deg_prec) + ',' + str(v_factor0)
            f.write(allangle + '\n')
            f.close()
    sys.stdout = orig_stdout
    out_f.close()

In [ ]:
filter_name='F410M'
filename_list='F410M_oldcat_filenames.txt' 
path_to_data='/grp/hst/wfc3h/verap/CAL_14393/F410M_oldcat/xymc/' 
path_to_coeff_files='/grp/hst/wfc3h/verap/CAL_14393/F410M_oldcat/meta_pix.v3/'

make_idctab_main(filter_name, filename_list, path_to_data, path_to_coeff_files)